In [ ]:
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import wiener
import glob
import os
import subprocess

In [ ]:
def wiener_filter(input_file, output_file):
    print('Begin filtering file ' + input_file)
    
    # inputing the file
    sar_image=gdal.Open(input_file)
    sar_band=sar_image.GetRasterBand(1)
    sar_band.GetMetadata()
    img_array=sar_band.ReadAsArray()
    [cols, rows]=img_array.shape

    filtered_image = wiener(img_array, 5)  #second parameter is window size, third parameter(unspecified) is the noise-power, if not specified it takes the local variance into account
    
    # writing output file
    driver = gdal.GetDriverByName("GTiff")
    output = driver.Create(output_file, rows, cols, 1, gdal.GDT_Float32)
    proj = output.SetGeoTransform(sar_image.GetGeoTransform())  
    output.SetProjection(sar_image.GetProjection())
    output.SetGCPs(sar_image.GetGCPs(), "4326")
    geoband = output.GetRasterBand(1)
    geoband.WriteArray(filtered_image)
    output.GetRasterBand(1).SetNoDataValue(-9999)
    output.FlushCache()
    output = None
    band=None
    print('Created file ' + output_file)
    
    print('End filtering file ' + input_file)    

In [ ]:
for input_file in glob.glob('/home/cristina/Seeps/Workflow/Filters/ROI/*/NRCS/*tif'):
    output_file = input_file.split(os.sep)
    output_file[-1] = output_file[-1][:-4] + '_Wiener_5.tif'
    output_file = output_file[:-2] + ['Wiener'] + output_file[-2:]
    output_file = os.sep.join(output_file)
    wiener_filter(input_file, output_file)